# Convert variables stored in JSON files to LaTeX variables

In [1]:
from IPython.display import display
from pathlib import Path
import json
import numpy as np
import xarray as xr

from boost_and_layouts import (
    create_latex_variable,
    create_loglist,
    list_to_latex,
    format_variable_name,
)

In [2]:
def parameters_to_latex(parameters, survey):
    """
    Generate a list of latex variables from a set of parameters
    """
    parameters_to_latex = []

    for layout in parameters:
        for depth_type in parameters[layout]:
            for parameter in parameters[layout][depth_type]:

                if parameter == "depth_type":
                    continue
                elif parameter == "damping":
                    values = create_loglist(parameters[layout][depth_type][parameter])
                else:
                    values = list_to_latex(parameters[layout][depth_type][parameter])
                variable_name = format_variable_name(
                    "_".join([survey, layout, depth_type, parameter])
                )
                parameters_to_latex.append(
                    r"\newcommand{{\{variable_name}}}{{{values}}}".format(
                        variable_name=variable_name, values=values
                    )
                )
    return parameters_to_latex


def format_damping(variable_name, value):
    """
    Convert damping to a LaTeX variable
    """
    variable_name = format_variable_name(variable_name)
    value = "e{:.0f}".format(np.log10(value))
    return r"\newcommand{{\{variable_name}}}{{\num{{{value}}}}}".format(
        variable_name=variable_name, value=value
    )


def best_parameters_to_latex(parameters, survey):
    """
    Convert best parameters to LaTeX variables

    Parameters
    ----------
    parameters : dict
        Dictionary containing the parameters of the best prediction.
    survey : str
        Name of the gridded survey. Eg. ``"ground"``, ``"airborne"``.
    """
    latex_variables = []
    layout = parameters["layout"]
    depth_type = parameters["depth_type"]
    for key, value in parameters.items():
        if key in ["metadata", "depth_type", "layout"]:
            continue

        variable_name = "_".join(["best", survey, layout, depth_type, key])
        if key == "damping":
            variable = format_damping(variable_name, value)
        elif key == "rms":
            variable = create_latex_variable(variable_name, value, fmt=".2f")
        else:
            variable = create_latex_variable(variable_name, value)
        latex_variables.append(variable)
    return latex_variables

/home/santi/.anaconda3/envs/eql-gradient-boosted/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Define results and manuscript directories

In [3]:
results_dir = Path("..") / "results"
manuscript_dir = Path("..") / "manuscript"
ground_results_dir = results_dir / "ground_survey"
airborne_results_dir = results_dir / "airborne_survey"

## Source layouts schematics

In [4]:
json_file = results_dir / "source-layouts-schematics.json"
tex_file = manuscript_dir / "source-layouts-schematics.tex"

with open(json_file, "r") as f:
    variables = json.loads(f.read())

In [5]:
latex_variables = [
    create_latex_variable(key, value) for key, value in variables.items()
]
with open(tex_file, "w") as f:
    f.write("\n".join(latex_variables))

## Synthetic survey and target grid

In [6]:
json_file = results_dir / "synthetic-surveys.json"
tex_file = manuscript_dir / "synthetic-surveys.tex"

with open(json_file, "r") as f:
    variables = json.loads(f.read())

In [7]:
units = variables.copy()

units["n_prisms"] = None
units["model_easting"] = "meter"
units["model_northing"] = "meter"
units["model_depth"] = "meter"
units["model_min_density"] = "kg per cubic meter"
units["model_max_density"] = "kg per cubic meter"
units["survey_easting"] = "meter"
units["survey_northing"] = "meter"
units["survey_noise"] = "milli Gal"
units["ground_survey_points"] = None
units["ground_survey_min_height"] = "meter"
units["ground_survey_max_height"] = "meter"
units["airborne_survey_points"] = None
units["airborne_survey_min_height"] = "meter"
units["airborne_survey_max_height"] = "meter"
units["target_height"] = "meter"
units["target_spacing"] = "meter"
units["target_easting_size"] = None
units["target_northing_size"] = None

In [8]:
latex_variables = [
    create_latex_variable(key, value, unit=units[key])
    for key, value in variables.items()
]
display(latex_variables)

['\\newcommand{\\NPrisms}{64}',
 '\\newcommand{\\ModelEasting}{\\SI{111319}{\\meter}}',
 '\\newcommand{\\ModelNorthing}{\\SI{111319}{\\meter}}',
 '\\newcommand{\\ModelDepth}{\\SI{10000}{\\meter}}',
 '\\newcommand{\\ModelMinDensity}{\\SI{-900}{\\kg \\per \\cubic \\meter}}',
 '\\newcommand{\\ModelMaxDensity}{\\SI{500}{\\kg \\per \\cubic \\meter}}',
 '\\newcommand{\\SurveyEasting}{\\SI{111319}{\\meter}}',
 '\\newcommand{\\SurveyNorthing}{\\SI{110576}{\\meter}}',
 '\\newcommand{\\SurveyNoise}{\\SI{1}{\\milli \\Gal}}',
 '\\newcommand{\\GroundSurveyPoints}{963}',
 '\\newcommand{\\GroundSurveyMinHeight}{\\SI{0}{\\meter}}',
 '\\newcommand{\\GroundSurveyMaxHeight}{\\SI{2052.2}{\\meter}}',
 '\\newcommand{\\AirborneSurveyPoints}{5673}',
 '\\newcommand{\\AirborneSurveyMinHeight}{\\SI{359}{\\meter}}',
 '\\newcommand{\\AirborneSurveyMaxHeight}{\\SI{1255}{\\meter}}',
 '\\newcommand{\\TargetHeight}{\\SI{2000}{\\meter}}',
 '\\newcommand{\\TargetSpacing}{\\SI{2}{\\meter}}',
 '\\newcommand{\\TargetEastin

In [9]:
with open(tex_file, "w") as f:
    f.write("\n".join(latex_variables))

## Gridding ground survey

In [10]:
json_file = results_dir / "parameters-ground-survey.json"
tex_file = manuscript_dir / "parameters-ground-survey.tex"

with open(json_file, "r") as f:
    parameters = json.loads(f.read())

In [11]:
latex_variables = parameters_to_latex(parameters, survey="ground")
display(latex_variables)

['\\newcommand{\\GroundSourceBelowDataConstantDepthDamping}{\\num{e-4}, \\num{e-3},$\\dots$, \\num{e2}}',
 '\\newcommand{\\GroundSourceBelowDataConstantDepthDepth}{\\numrange{1000}{17000}, step size \\num{2000}}',
 '\\newcommand{\\GroundSourceBelowDataRelativeDepthDamping}{\\num{e-4}, \\num{e-3},$\\dots$, \\num{e2}}',
 '\\newcommand{\\GroundSourceBelowDataRelativeDepthDepth}{\\numrange{1000}{17000}, step size \\num{2000}}',
 '\\newcommand{\\GroundSourceBelowDataVariableDepthDamping}{\\num{e-4}, \\num{e-3},$\\dots$, \\num{e2}}',
 '\\newcommand{\\GroundSourceBelowDataVariableDepthDepthFactor}{\\numlist{0.1;0.5;1;2;3;4;5;6}}',
 '\\newcommand{\\GroundSourceBelowDataVariableDepthDepth}{\\numrange{0}{1400}, step size \\num{200}}',
 '\\newcommand{\\GroundSourceBelowDataVariableDepthKNearest}{\\numlist{1;5;10;15}}',
 '\\newcommand{\\GroundBlockAveragedSourcesConstantDepthDamping}{\\num{e-4}, \\num{e-3},$\\dots$, \\num{e2}}',
 '\\newcommand{\\GroundBlockAveragedSourcesConstantDepthDepth}{\\numr

In [12]:
with open(tex_file, "w") as f:
    f.write("\n".join(latex_variables))

### Ground survey: best predictions

In [13]:
best_predictions = [
    xr.open_dataset(f)
    for f in ground_results_dir.iterdir()
    if "best_predictions" in f.name
]

latex_variables = []
for dataset in best_predictions:
    for array in dataset:
        latex_variables.extend(
            best_parameters_to_latex(dataset[array].attrs, survey="ground")
        )

tex_file = manuscript_dir / "best-parameters-ground-survey.tex"
with open(tex_file, "w") as f:
    f.write("\n".join(latex_variables))

## Gridding airborne survey

In [14]:
json_file = results_dir / "parameters-airborne-survey.json"
tex_file = manuscript_dir / "parameters-airborne-survey.tex"

with open(json_file, "r") as f:
    parameters = json.loads(f.read())

In [15]:
latex_variables = parameters_to_latex(parameters, survey="airborne")
display(latex_variables)

['\\newcommand{\\AirborneSourceBelowDataConstantDepthDamping}{\\num{e-4}, \\num{e-3},$\\dots$, \\num{e2}}',
 '\\newcommand{\\AirborneSourceBelowDataConstantDepthDepth}{\\numrange{1000}{17000}, step size \\num{2000}}',
 '\\newcommand{\\AirborneSourceBelowDataRelativeDepthDamping}{\\num{e-4}, \\num{e-3},$\\dots$, \\num{e2}}',
 '\\newcommand{\\AirborneSourceBelowDataRelativeDepthDepth}{\\numrange{1000}{17000}, step size \\num{2000}}',
 '\\newcommand{\\AirborneSourceBelowDataVariableDepthDamping}{\\num{e-4}, \\num{e-3},$\\dots$, \\num{e2}}',
 '\\newcommand{\\AirborneSourceBelowDataVariableDepthDepthFactor}{\\numrange{1}{6}, step size \\num{1}}',
 '\\newcommand{\\AirborneSourceBelowDataVariableDepthDepth}{\\numrange{50}{1450}, step size \\num{200}}',
 '\\newcommand{\\AirborneSourceBelowDataVariableDepthKNearest}{\\numlist{1;5;10;15}}',
 '\\newcommand{\\AirborneBlockAveragedSourcesConstantDepthDamping}{\\num{e-4}, \\num{e-3},$\\dots$, \\num{e2}}',
 '\\newcommand{\\AirborneBlockAveragedSource

In [16]:
with open(tex_file, "w") as f:
    f.write("\n".join(latex_variables))

### Airborne survey: best predictions

In [17]:
best_predictions = [
    xr.open_dataset(f)
    for f in airborne_results_dir.iterdir()
    if "best_predictions" in f.name
]

latex_variables = []
for dataset in best_predictions:
    for array in dataset:
        latex_variables.extend(
            best_parameters_to_latex(dataset[array].attrs, survey="airborne")
        )

tex_file = manuscript_dir / "best-parameters-airborne-survey.tex"
with open(tex_file, "w") as f:
    f.write("\n".join(latex_variables))

## Gradient boosted sources

In [18]:
json_file = results_dir / "boost-overlapping.json"
tex_file = manuscript_dir / "boost-overlapping.tex"

with open(json_file, "r") as f:
    variables = json.loads(f.read())

units["boost_overlapping_window_size"] = "meter"

In [19]:
latex_variables = [
    create_latex_variable(key, value, unit=units[key])
    for key, value in variables.items()
]
display(latex_variables)

['\\newcommand{\\BoostOverlappingWindowSize}{\\SI{30000}{\\meter}}']

In [20]:
with open(tex_file, "w") as f:
    f.write("\n".join(latex_variables))

## Australia gravity

In [21]:
json_file = results_dir / "australia.json"
tex_file = manuscript_dir / "australia.tex"

with open(json_file, "r") as f:
    variables = json.loads(f.read())

variables["australia_eql_window_size"] *= 1e-3

units["australia_eql_depth"] = "meter"
units["australia_eql_damping"] = None
units["australia_eql_spacing"] = "meter"
units["australia_eql_window_size"] = "kilo meter"
units["australia_eql_n_sources"] = None

In [22]:
latex_variables = [
    create_latex_variable(key, value, unit=units[key])
    for key, value in variables.items()
]
display(latex_variables)

['\\newcommand{\\AustraliaEqlDepth}{\\SI{4000}{\\meter}}',
 '\\newcommand{\\AustraliaEqlDamping}{100}',
 '\\newcommand{\\AustraliaEqlSpacing}{\\SI{1800}{\\meter}}',
 '\\newcommand{\\AustraliaEqlWindowSize}{\\SI{225}{\\kilo \\meter}}',
 '\\newcommand{\\AustraliaEqlNSources}{796744}']

In [23]:
with open(tex_file, "w") as f:
    f.write("\n".join(latex_variables))